Passing tools to chat models
将工具传递给聊天模型

支持工具调用功能的聊天模型实现一个 .bind_tools 方法，该方法接收函数、Pydantic 模型或 LangChain 工具对象的列表，
并以预期格式将它们绑定到聊天模型。聊天模型的后续调用将在其对 LLM.

下面我们实现简单的算术工具：

In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

ms_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('MOONSHOT_API_BASE'),
    openai_api_key=os.getenv('MOONSHOT_API_KEY'),
    model_name="moonshot-v1-8k",
    temperature=0,
    streaming=True,
)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3-8b-instruct",
    temperature=0,
    streaming=True,
)

groq_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('GROQ_API_BASE'),
    openai_api_key=os.getenv('GROQ_API_KEY'),
    model_name="llama3-8b-8192",
    temperature=0,
    streaming=True,
)

In [ ]:
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

LangChain还实现了一个 @tool 装饰器，允许进一步控制工具模式，例如工具名称和参数描述。有关详细信息，请参阅此处的操作指南。

我们还可以使用 Pydantic 定义模式：

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

In [3]:
llm_with_tools = qw_llm_openai.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content="The result of multiplying 3 by 12 is 36. Let's calculate it.", additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_a8c6480217cb4a7fb2cd49', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen2-1.5b-instruct'}, id='run-e5589ba1-d44c-4dc0-b622-e8c16569103c-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_a8c6480217cb4a7fb2cd49', 'type': 'tool_call'}])

In [4]:
query = "What is 3 * 12? Also, what is 11 + 49?"
# 如果LLM响应中包含工具调用，则它们将作为属性中 .tool_calls 的工具调用对象列表附加到相应的消息或消息块中。
# A ToolCall 是一个类型化字典，包括工具名称、参数值字典和（可选）标识符。没有工具调用的消息默认为此属性的空列表。
llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_ea348dcdde8d4556840ab4',
  'type': 'tool_call'}]

该 .tool_calls 属性应包含有效的工具调用。请注意，有时，模型提供程序可能会输出格式错误的工具调用（例如，无效 JSON 的参数）。
在这些情况下分析失败时，InvalidToolCall 的实例将填充到属性中 .invalid_tool_calls 。
可以 InvalidToolCall 有名称、字符串参数、标识符和错误消息。

如果需要，输出分析器可以进一步处理输出。例如，我们可以使用 PydanticToolsParser 将 .tool_calls 属性上填充的现有值转换回原始 Pydantic 类：

In [5]:
from langchain_core.output_parsers import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
chain.invoke(query)

[Multiply(a=3, b=12)]